## Installation librairies

In [2]:
!pip install openai==0.28
!pip install langchain
!pip install tiktoken
!pip install faiss-cpu
!pip install PyPDF2
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


## Librairies

In [3]:
#from openai import OpenAI
import openai
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
#from langchain.llms import OpenAI
#from langchain.llms import OpenAIChat
from langchain_community.chat_models import ChatOpenAI

In [9]:
# Configuration de l'API OpenAI
openai.api_key = "PUT YOUR KEY HERE"


In [10]:
def simple_llm_call(prompt):
    try:
        # Effectuer un appel à l'API OpenAI avec le modèle GPT-3.5
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,  # Limite de tokens pour la réponse
            temperature=0.7, # Contrôle la créativité (0.0 à 1.0)
        )

        # Récupérer et retourner la réponse du modèle
        return response.choices[0].message['content'].strip()

    except Exception as e:
        return f"Erreur lors de l'appel à l'API : {str(e)}"

In [11]:
our_query = "how many awards did Huhu won ?"
print(simple_llm_call(our_query))

I'm sorry, but I don't have specific information on an individual named "Huhu" and their awards. If you can provide more context or details, I may be able to assist you further.


## Simple LLM call with generic knowledge

## LLM call with Pdf reference

In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/legatech/

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/legatech


In [13]:
data = PdfReader("toto.pdf")
combined_text = ""
for i,page in enumerate(data.pages):
  text = page.extract_text()
  if text:
    combined_text += text

In [14]:
combined_text

'Huhu is a fictional character, he has very good humor.  \nAll of his friend like him a lot, Huhu is from Egy pt. \nHuhu has won a total of 5 awords.  '

In [15]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 200,
    chunk_overlap = 200,
    length_function = len
)
find_data = text_splitter.split_text(combined_text)

In [16]:
find_data

['Huhu is a fictional character, he has very good humor.  \nAll of his friend like him a lot, Huhu is from Egy pt. \nHuhu has won a total of 5 awords.']

In [17]:
len(find_data)

1

In [18]:
embeddings = OpenAIEmbeddings()

<ipython-input-18-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [19]:
documenTsearch = FAISS.from_texts(find_data,embeddings)

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

# Chargement du modèle GPT-3.5-turbo via ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Création de la chaîne question/réponse
chain = load_qa_chain(llm, chain_type="stuff")

<ipython-input-20-e781f022213c>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
<ipython-input-20-e781f022213c>:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answe

In [21]:
our_query = "how many awards did Huhu won ?"
docs = documenTsearch.similarity_search(our_query)
chain.run(input_documents=docs, question=our_query)

<ipython-input-21-f0ea6bdba091>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=our_query)


'Huhu has won a total of 5 awards.'